
# Numerical Thin-Shell Shock in a Broken CSM

This example demonstrates how to use the
:class:`~dynamics.supernovae.shock_dynamics.NumericalThinShellShockEngine`
to model the evolution of a supernova shock expanding into a **non-uniform
circumstellar medium (CSM)**.

Unlike self-similar shock solutions, which rely on idealized power-law density
profiles, the numerical thin-shell engine allows for **arbitrary ejecta and CSM
density structures**. This makes it well-suited for modeling realistic progenitor
environments, such as winds with termination shocks or multi-phase media.

In this example, we consider:

- Supernova ejecta with a broken power-law density profile in velocity space
- A circumstellar medium consisting of:
  - a wind-like $\rho \propto r^{-2}$ profile close to the progenitor
  - a transition to a uniform-density interstellar medium (ISM) at larger radii

We then compute and visualize the evolution of the shock radius, velocity,
and swept-up mass as a function of time.


## Imports

We begin by importing the relevant numerical, plotting, and Triceratops
utilities needed for this example.



In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from astropy import units as u

from triceratops.dynamics.supernovae.profiles import (
    get_broken_power_law_ejecta_kernel_func,
)
from triceratops.dynamics.supernovae.shock_dynamics import NumericalThinShellShockEngine
from triceratops.utils.plot_utils import set_plot_style

## Physical Setup

We now define the physical parameters of the system. These values are chosen
to be broadly representative of a core-collapse supernova interacting with
its surrounding environment.



In [ ]:
# Mass-loss rate of the progenitor wind
M_dot = 1e-5 * u.M_sun / u.yr

# Wind velocity
v_wind = 1e5 * u.cm / u.s

# Duration of the wind phase prior to explosion
t_wind = 1e8 * u.s

# Density of the ambient ISM
rho_ism = 1e-21 * u.g / u.cm**3

# Ejecta properties
E_ej = 1e48 * u.erg
M_ej = 10 * u.M_sun

# Radius of the wind termination shock
R_wind = v_wind * t_wind

## Circumstellar Medium Density Profile

The numerical thin-shell engine requires the CSM density profile to be provided
as a **callable function** returning the density in CGS units.

Here we construct a broken profile:

- For $r < R_{\\rm wind}$: a steady wind with $\\rho \\propto r^{-2}$
- For $r \\ge R_{\\rm wind}$: a uniform ISM



In [ ]:
rho_0_cgs = (M_dot / (4 * np.pi * R_wind**2 * v_wind)).to_value(u.g / u.cm**3)
R_wind_cgs = R_wind.to_value(u.cm)
rho_ISM_cgs = rho_ism.to_value(u.g / u.cm**3)


def rho_csm(r):
    """
    Broken CSM density profile.

    Parameters
    ----------
    r : float or ndarray
        Radius in cm.

    Returns
    -------
    rho : float or ndarray
        CSM density in g/cm^3.
    """
    return np.where(
        r < R_wind_cgs,
        rho_0_cgs * r**-2,
        rho_ISM_cgs,
    )

## Ejecta Density Profile

For homologously expanding ejecta, the density takes the form

\begin{align}\\rho_{\\rm ej}(r, t) = t^{-3} G_{\\rm ej}(r / t)\end{align}

Rather than specifying $\\rho(r, t)$ directly, the thin-shell engine
requires the kernel function $G_{\\rm ej}(v)$.

Triceratops provides helper functions for constructing commonly used ejecta
profiles. Here we adopt a Chevalier-style broken power-law profile.



In [ ]:
G_ej = get_broken_power_law_ejecta_kernel_func(
    E_ej,
    M_ej,
    n=10,
    delta=0,
)

## Shock Engine Initialization

The shock engine itself is stateless. All physical information is passed in
at evaluation time.



In [ ]:
shock_engine = NumericalThinShellShockEngine()

## Initial Conditions

Numerical integration of the thin-shell equations requires initial conditions
for the shock radius, velocity, time, and swept-up mass.



In [ ]:
params = {
    "rho_csm": rho_csm,
    "G_ej": G_ej,
    "R_0": 1e10 * u.cm,
    "v_0": 1e9 * u.cm / u.s,
    "t_0": 1e1 * u.s,
    "M_0": 1e-4 * u.M_sun,
}

## Time Grid

We evaluate the shock evolution over several orders of magnitude in time.



In [ ]:
time = np.geomspace(1, 1000, 500) * u.day

## Shock Evolution

We now compute the shock properties using the high-level API, which handles
unit conversion and validation internally.



In [ ]:
shock_properties = shock_engine.compute_shock_properties(
    time,
    **params,
)

## Comparison: Homologous Expansion

For reference, we compute the radius expected from free homologous expansion
at the initial shock velocity.



In [ ]:
R_homologous = (1e9 * u.cm / u.s) * time

## Visualization

Finally, we visualize the evolution of the shock radius, velocity, and
swept-up mass.



In [ ]:
set_plot_style()

fig, axes = plt.subplots(3, 1, figsize=(8, 10), sharex=True)

# Shock radius
axes[0].loglog(
    time.to_value(u.day),
    shock_properties["radius"].to_value(u.cm),
    label="Numerical Shock Radius",
)
axes[0].loglog(
    time.to_value(u.day),
    R_homologous.to_value(u.cm),
    ls="--",
    label="Homologous Expansion",
)
axes[0].set_ylabel("Shock Radius (cm)")
axes[0].legend()
axes[0].grid(True, which="both", ls="--", alpha=0.5)

# Shock velocity
axes[1].loglog(
    time.to_value(u.day),
    shock_properties["velocity"].to_value(u.cm / u.s),
)
axes[1].set_ylabel("Shock Velocity (cm/s)")
axes[1].grid(True, which="both", ls="--", alpha=0.5)

# Swept-up mass
axes[2].loglog(
    time.to_value(u.day),
    shock_properties["mass"].to_value(u.M_sun),
)
axes[2].set_xlabel("Time (days)")
axes[2].set_ylabel(r"Swept-up Mass ($M_\odot$)")
axes[2].grid(True, which="both", ls="--", alpha=0.5)

plt.tight_layout()
plt.show()